<a href="https://colab.research.google.com/github/Bmiguez/Arezzo-Valuation/blob/main/Valuation_Arezzo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# Importando inicialmente as bibliotecas necessárias
import pandas as pd
import numpy as np
import statsmodels.api as sm
import yfinance as yf
import sys
import datetime
from dateutil.relativedelta import relativedelta

In [21]:
# Definindo ticker e índice de referencia
ticker = "ARZZ3.SA"
indice = "^BVSP"

# Definindo a data de referência e o intervalo de dados
ano = 2023
mes = 1
dia = 1

data_ref = datetime.date(ano, mes, dia)
data_ini = data_ref - relativedelta(months = 46)

In [22]:
from ast import increment_lineno
# Extrair retornos ajustados da empresa e do benchmark

retornos = yf.download([ticker, indice], start = data_ini, end = data_ref, interval="1mo")["Adj Close"].pct_change()

retornos = pd.DataFrame(retornos.dropna())

y = retornos[ticker]
x = retornos[indice]



[*********************100%***********************]  2 of 2 completed


In [23]:
# Começando a regressão

x = sm.add_constant(x)

modelo = sm.OLS(y, x)

resultados = modelo.fit()

print(resultados.summary())
# Extraindo valor do Beta

beta = resultados.params[1]

conf_interval = resultados.conf_int(alpha=0.05)

lower_bound = conf_interval.iloc[1, 0]
upper_bound = conf_interval.iloc[1, 1]

print("Intervalo de Confiança:")
print("Limite Inferior:", lower_bound)
print("Limite Superior:", upper_bound)


                            OLS Regression Results                            
Dep. Variable:               ARZZ3.SA   R-squared:                       0.551
Model:                            OLS   Adj. R-squared:                  0.541
Method:                 Least Squares   F-statistic:                     52.80
Date:                Tue, 20 Jun 2023   Prob (F-statistic):           5.30e-09
Time:                        13:17:32   Log-Likelihood:                 55.153
No. Observations:                  45   AIC:                            -106.3
Df Residuals:                      43   BIC:                            -102.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0106      0.011      0.978      0.3

In [24]:
# Cálculo do Custo do equity

# Desalavancando beta

tax = 0.27
debt = 1060093
equity = 2716661

unlev_beta = beta/(1+(1-tax)*debt/equity)

# Risk Free Obtido 14/06
rf = 0.1096

# Default spread 14/06
dfs = 0.0314



# Calculando equity premium

data_ini = data_ref - relativedelta(years = 5)

retornos = yf.download(indice, start = data_ini, end = data_ref, interval="1mo")["Adj Close"].pct_change()

retornos = retornos.dropna()

retorno_anualizado = (float(retornos.mean())+1)**12-1
print(retorno_anualizado)

[*********************100%***********************]  1 of 1 completed
0.08755353683124478


In [25]:
path = "https://github.com/Bmiguez/Arezzo-Valuation/raw/main/Dados_Valuation.xlsx"
aba_bp = "BP"
aba_dre = "DRE"
aba_fc = "FC"
aba_lp = "LojasProp"
aba_indicadores = "Indicadores"
aba_receitabruta = "Receita Bruta"

#### Criação dos Dataframes
#### Balanço Patrimonial
df_bp = pd.read_excel(path, aba_bp)

#### Demonstração Resultado do Exercício
df_dre = pd.read_excel(path, aba_dre)

#### Fluxo de Caixa
df_fc = pd.read_excel(path, aba_fc)

#### Lojas Próprias
df_lp = pd.read_excel(path, aba_lp)

#### Indicadores
df_ind = pd.read_excel(path, aba_indicadores)

#### Receita Bruta
df_receita = pd.read_excel(path, aba_receitabruta)

In [26]:
# Calculando a média das deduções entre receita bruta e liquida
receita_bruta = df_receita.iloc[:,1]
receita_liquida = df_dre.iloc[:,1]

deducoes = pd.DataFrame({"Ano": range(2010,2023), "Percentual":receita_liquida/receita_bruta})

## Calculando a média das deduções nos últimos quatro anos
deducoes["Média_5_anos"] = deducoes["Percentual"].rolling(5).mean().shift()



### Formulando premissas de receita com base no numero de lojas

df_lojas_prop = df_lp.iloc[:,[0] + list(range(15,25))].apply(pd.to_numeric, errors='coerce')

franquias = df_lp.iloc[:,[0] + list(range(6,14))].apply(pd.to_numeric, errors='coerce')

## Cálculo do número de aberturas de lojas prórprias por marca
aberturas_LP = df_lojas_prop.iloc[:,1:].diff()
aberturas_LP = aberturas_LP.reindex(columns=["Ano"] + aberturas_LP.columns[:-1].tolist())
aberturas_LP["Ano"] = df_lojas_prop.iloc[:, 0]
aberturas_LP = aberturas_LP.drop(aberturas_LP.index[0])

# Cálculo da média dos últimos 3 anos destas aberturas

df_mediaslp_aberturas = aberturas_LP.iloc[:,1:].rolling(5).mean().shift()
df_mediaslp_aberturas = df_mediaslp_aberturas.reindex(columns=["Ano"] + df_mediaslp_aberturas.columns[:-1].tolist())
df_mediaslp_aberturas["Ano"] = df_lojas_prop.iloc[:, 0]


## Cálculo do número de aberturas de franquias por marca

aberturas_FR = franquias.iloc[:,1:].diff()
aberturas_FR = aberturas_FR.reindex(columns=["Ano"] + aberturas_FR.columns[:-1].tolist())
aberturas_FR["Ano"] = franquias.iloc[:, 0]
aberturas_FR = aberturas_FR.drop(aberturas_FR.index[0])

# Cálculo da média dos últimos 5 anos destas aberturas

df_mediasfr_aberturas = aberturas_FR.iloc[:,1:].rolling(5).mean().shift()
df_mediasfr_aberturas = df_mediasfr_aberturas.reindex(columns=["Ano"] + df_mediasfr_aberturas.columns[:-1].tolist())
df_mediasfr_aberturas["Ano"] = df_lojas_prop.iloc[:, 0]


# Cálculo do mix de receita por canal por ano
canais = ["Franquias", "Multimarcas", "Lojas próprias", "Web Commerce", "Outros²"]

df_canais = df_receita.loc[:,canais]
df_particip_canais = df_canais.apply(lambda row: row / row.sum(), axis=1)

### Cálculo da receita por loja/receita por franquia
## Obter receita de lojas e receita de franquias de cada marca

#Arezzo

receitalp_arezzo = df_receita.loc[:,"Arezzo"] * df_particip_canais.loc[:,"Lojas próprias"]
receitafr_arezzo = df_receita.loc[:,"Arezzo"] * df_particip_canais.loc[:,"Franquias"]

#Schutz

receitalp_schutz = df_receita.loc[:,"Schutz"] * df_particip_canais.loc[:,"Lojas próprias"]
receitafr_schutz = df_receita.loc[:,"Schutz"] * df_particip_canais.loc[:,"Franquias"]

#Anacapri
receitalp_anacapri = df_receita.loc[:,"Anacapri"] * df_particip_canais.loc[:,"Lojas próprias"]
receitafr_anacapri = df_receita.loc[:,"Anacapri"] * df_particip_canais.loc[:,"Franquias"]

#AR&CO
receitalp_arco = df_receita.loc[:,"AR&CO"] * df_particip_canais.loc[:,"Lojas próprias"]
receitafr_arco = df_receita.loc[:,"AR&CO"] * df_particip_canais.loc[:,"Franquias"]

#Outros
receitalp_outros = df_receita.loc[:,"Outros¹"] * df_particip_canais.loc[:,"Lojas próprias"]
receitafr_outros = df_receita.loc[:,"Outros¹"] * df_particip_canais.loc[:,"Franquias"]

## Receita por lojas
print (df_lojas_prop.loc[1:,"Arezzo.1"].reset_index(drop=True))

rec_porLP_arr= receitalp_arezzo / df_lojas_prop.loc[1:,"Arezzo.1"].reset_index(drop=True)
rec_porFR_arr= receitafr_arezzo / franquias.loc[1:,"Arezzo"].reset_index(drop=True)

rec_porLP_sch= receitalp_schutz / df_lojas_prop.loc[1:,"Schutz.1"].reset_index(drop=True)
rec_porFR_sch= receitafr_schutz / franquias.loc[1:,"Schutz"].reset_index(drop=True)

rec_porLP_arc= receitalp_arco / df_lojas_prop.loc[1:,"AR&CO.1"].reset_index(drop=True)
rec_porFR_arc= receitafr_arco / franquias.loc[1:,"AR&CO"].reset_index(drop=True)

rec_porLP_ana= receitalp_anacapri / df_lojas_prop.loc[1:,"Anacapri.1"].reset_index(drop=True)
rec_porFR_ana= receitafr_anacapri / franquias.loc[1:,"Anacapri"].reset_index(drop=True)

rec_porLP_out= receitalp_outros / df_lojas_prop.loc[1:,["Alexandre Birman"] + ["Fiever.1"] + ["Alme.1"] + ["Vans.1"] + ["Carol Bassi"] + ["Vicenza"]].sum(axis=1).reset_index(drop=True)
rec_porFR_out= receitafr_outros / franquias.loc[1:,["Fiever"] + ["Alme"] + ["Vans"] + ["My Shoes"]].sum(axis=1).reset_index(drop=True)

### Cálculo da média da receita de webcommerce e de multimarcas

rec_web = df_canais.loc[:,"Web Commerce"]
rec_mult = df_canais.loc[:,"Multimarcas"]
rec_out = df_canais.loc[:,"Outros²"]

proj_rec_web = rec_web.pct_change().rolling(5).mean().shift()
proj_rec_mult = rec_mult.pct_change().rolling(5).mean().shift()
proj_rec_out = rec_out.rolling(5).mean().shift()


0     13
1     19
2     19
3     17
4     19
5     15
6     15
7     15
8     14
9     10
10    12
11    18
12    18
Name: Arezzo.1, dtype: int64


In [34]:
### Organizando projeções

## Número de lojas

total_FR = df_lp.loc[1:,"Número de franquias"].reset_index(drop=True)
total_LP = df_lp.loc[1:,"Número de lojas próprias"].reset_index(drop=True)
total_mult = df_lp.loc[1:,"Multimarca MI (sem overlap)"].reset_index(drop=True)

abert_FR = df_lp.loc[:,"Número de franquias"].diff().dropna().reset_index(drop=True)
abert_LP = df_lp.loc[:,"Número de lojas próprias"].diff().dropna().reset_index(drop=True)
abert_mult = df_lp.loc[:,"Multimarca MI (sem overlap)"].diff().dropna().reset_index(drop=True)

## Aberturas por marca
abertura_marcaLP = df_lp.iloc[1:,15:25].diff().dropna().reset_index(drop=True)
abertura_marcaFR = df_lp.iloc[1:,6,14].diff().dropna().reset_index(drop=True)



## Receita por canal

receita_LP = df_canais.loc[:,"Lojas próprias"]
receita_FR = df_canais.loc[:,"Franquias"]
receita_mult = df_canais.loc[:,"Multimarcas"]
receita_web = df_canais.loc[:,"Web Commerce"]
receita_out = df_canais.loc[:,"Outros²"]

## Receita por unidade

receita_por_LP = receita_LP/ total_LP
receita_por_FR = receita_FR / total_FR
receita_por_mult = receita_mult / total_mult

## Crescimento receita por unidade + receita ecom

var_por_LP = receita_por_LP.pct_change()
var_por_FR = receita_por_FR.pct_change()
var_por_mult = receita_por_mult.pct_change()
var_web = receita_web.pct_change()

# Projeções de crescimento de receita

proj_rec_LP = (var_por_LP[12] + var_por_LP[9] + var_por_LP[8] + var_por_LP[7] + var_por_LP[6])/5
proj_rec_FR = (var_por_FR[12] + var_por_FR[9] + var_por_FR[8] + var_por_FR[7] + var_por_FR[6])/5
proj_rec_mult = (var_por_mult[12] + var_por_mult[9] + var_por_mult[8] + var_por_mult[7] + var_por_mult[6])/5
proj_rec_out = (receita_out[12] + receita_out[9] + receita_out[8] + receita_out[7] + receita_out[6])/5
proj_rec_web = (var_web[12] + var_web[9] + var_web[8] + var_web[7] + var_web[6])/5

# Projeções de número de unidades

proj_abert_FR = (abert_FR[12] + abert_FR[9] + abert_FR[8] + abert_FR[7] + abert_FR[6])/5
proj_abert_LP = (abert_LP[12] + abert_LP[9] + abert_LP[8] + abert_LP[7] + abert_LP[6])/5
proj_abert_mult = (abert_mult[12] + abert_mult[9] + abert_mult[8] + abert_mult[7] + abert_mult[6])/5


    Arezzo.1  Schutz.1  Alexandre Birman  Anacapri.1 Fiever.1 Alme.1  Vans.1  \
1         13        10                 1           5        -      -     NaN   
2         19        17                 1           8        -      -     NaN   
3         19        27                 2           8        -      -     NaN   
4         17        27                 2           8        -      -     NaN   
5         19        27                 2           5        -      -     NaN   
6         15        26                 2           4        1      -     NaN   
7         15        23                 3           4        3      -     NaN   
8         15        22                 4           3        4      -     NaN   
9         14        17                 4           3        5      2     NaN   
10        10        17                 6           3        5      3     NaN   
11        12        16                 8           5        2      2     7.0   
12        18        20                 8

1º Valuation

2015-2022

In [31]:
### Primeiro valuation - 2015 até 2022

## Projetando receita bruta
# Lojas por Loja própria
receita_lp_2023 = receita_por_LP[12] * proj_rec_LP
receita_lp_2024 = receita_lp_2023 * proj_rec_LP
receita_lp_2025 = receita_lp_2024 * proj_rec_LP
receita_lp_2026 = receita_lp_2025 * proj_rec_LP
receita_lp_2027 = receita_lp_2026 * proj_rec_LP

# Total Lojas própriras
print(abert_LP)

# Receita ano / num de lojas * cresc * novas lojas






0      7.0
1     16.0
2     11.0
3     -2.0
4     -1.0
5     -5.0
6      0.0
7      0.0
8     -3.0
9     -1.0
10    83.0
11    26.0
12    32.0
Name: Número de lojas próprias, dtype: float64
